In [28]:
try:
    from pulp import *
except:
    print("please install pulp package")

def final_convert(df):
    import numpy as np
    import pandas as pd
    
    # convert df(9x9) into dt_new(81x9)
    def convert(df):
        import numpy as np
        import pandas as pd
        columns = [i for i in range(1,10)]
        rows = [i for i in range(1,82)]
        temp = pd.DataFrame(np.zeros((81,9)),index = rows, columns = columns)
        for i in range(1,10):
            temp.loc[1+9*(i-1):i*9] = np.where(df==i,1,0)
        return temp
    
    # create function make sure each point has only one value
    def clean_each(df_new):
        #Create A
        columns = [i for i in range(1,730)]
        rows = [i for i in range(1,82)]
        A = pd.DataFrame(np.zeros((81,729)),index = rows, columns = columns)

        #Create b
        columns = [1]
        rows = [i for i in range(1,82)]
        b = pd.DataFrame(np.zeros((81,1)),index = rows, columns = columns)  

        for i in range(1,10): #columns
            for j in range(1,10): #rows
                result = []
                for m in range(1,10):
                    result.append(df_new.loc[j+9*(m-1),i])
                    col = 9*(j-1)+81*(m-1)+i
                    A.loc[9*(i-1)+j,col] = 1
                    b.loc[9*(i-1)+j] = 1                         
                if sum(result) == 0:
                    continue
                elif sum(result) != 1:
                    print('Error')
                else:      
                    index = result.index(1)+1
                    col = 9*(j-1)+81*(index-1)+i
                    A.loc[9*(i-1)+j,col] = 0
                    b.loc[9*(i-1)+j] = 0
        return A,b
    
    # functions make sure each row has only one value
    def clean_row(df_new):
        import numpy as np
        import pandas as pd
    #create row_check R
        columns = [i for i in range(1,730)]
        rows = [i for i in range(1,82)]
        R = pd.DataFrame(np.zeros((81,729)),index = rows, columns = columns)
    #creat row_check b    
        columns = [1]
        rows = [i for i in range(1,82)]
        b = pd.DataFrame(np.ones((81,1)),index = rows, columns = columns) 
    #check
        for i in range(1,82):
            result = []
            for j in range(1,10):
                result.append(df_new.loc[i,j])
                R.loc[i,9*(i-1)+j] = 1
            if sum(result) == 1:
                index = result.index(1)+1
                R.loc[i,9*(i-1)+index] = 0
                b.loc[i] = 0
        return R,b
    
    # functions make sure each column has only one value
    def clean_column(df):
        import numpy as np
        import pandas as pd
    #create row_check A
        columns = [i for i in range(1,730)]
        rows = [i for i in range(1,82)]
        C = pd.DataFrame(np.zeros((81,729)),index = rows, columns = columns)
    #creat row_check b    
        columns = [1]
        rows = [i for i in range(1,82)]
        b = pd.DataFrame(np.ones((81,1)),index = rows, columns = columns) 
    #check
        for i in range(0,9):
            for j in range(1,10):
                result = []
                for w in range(0,9):
                    result.append(df[j][i*9+1+w])
                    C.loc[i*9+j,j+81*i+9*w] = 1
                    if sum(result) ==1:
                        index = result.index(1)+1
                        C.loc[i*9+j,j+81*i+9*(index-1)]=0
                        b.loc[i*9+j]=0
        return C,b
    
    # functions make sure the sum of each 3 by 3 grid equals to 1
    def clean_grid(df_new):
        #rearrange the index of the dataset df_new
        df_new.index = [i for i in range(81)]
        df_new.columns = [i for i in range(9)]
        import numpy as np
        import pandas as pd
    #create an empty matrix G for grid check
        columns = [i for i in range(729)]
        rows = [i for i in range(81)]
        G = pd.DataFrame(np.zeros((81,729)),index = rows, columns = columns)
    #create an empty vertical array b 
        columns = [1]
        rows = [i for i in range(81)]
        b = pd.DataFrame(np.ones((81,1)),index = rows, columns = columns) 
    #check
        for i in range(81):#ith grid, and i is also the number of row of G and the number of constraint
            #The coordinates of the first element in the grid:
            col = (i % 3) *3
            row = (i // 3) * 3
            #now we creat a list containing the elements of the grid in df_new
            grid_list =list()
            for j in range(3):
                for k in range(3):
                    row_num = row + j
                    col_num = col + k
                    grid_list.append(df_new.loc[row_num, col_num])
                    # Also we let the coefficient of the corresponding item in G to be 1
                    G.loc[i,row_num * 9 + col_num] = 1
            #now check if all the elements in grid_list are zeros
            sum_list = sum(grid_list)
            if sum_list == 0:
                b.loc[i,1] = 1
                continue
            elif sum_list == 1:
                b.loc[i,1] = 0
                #we find out which element in sum_list equal to 1 and let corresponding coefficient in G to be 0
                index_one = grid_list.index(1)
                row_one = row + index_one // 3
                col_one = col + index_one % 3
                G.loc[i,row_one * 9 + col_one] = 0
        #To be compatible with the function defined above, we change the index of G and df_new back
        df_new.index = [i for i in range(1,82)]
        df_new.columns = [i for i in range(1,10)]
        G.index = [i for i in range(1,82)]
        G.columns = [i for i in range(1,730)]
        b.index = [i for i in range(1,82)]
        return G,b
    
    def convert_back(df_new):
        columns = [i for i in range(1,10)]
        temp = pd.DataFrame(np.zeros((9,9)),columns = columns)
        for i in range(1,10):
            temp = temp + i*df_new.loc[1+(i-1)*9:i*9].reset_index(drop=True)
        temp.index = [i for i in range(1,10)]
        return temp.astype(int)
    
    
    df_new = convert(df)
    A1, b1 = clean_each(df_new)
    A2, b2 = clean_row(df_new)
    A3, b3 = clean_column(df_new)
    A4, b4 = clean_grid(df_new)
    
    A1 = A1.values;A2 = A2.values;A3 = A3.values;A4 = A4.values
    A1_list = A1.tolist();A2_list = A2.tolist();A3_list = A3.tolist();A4_list =A4.tolist()
    A1_list.extend(A2_list);A1_list.extend(A3_list);A1_list.extend(A4_list)
    A = A1_list
    columns = [i for i in range(1,730)]
    rows = [i for i in range(1,325)]
    A = pd.DataFrame(A,index = rows, columns = columns)
    
    b1_list = b1.values.tolist();b2_list = b2.values.tolist();b3_list = b3.values.tolist();b4_list = b4.values.tolist()
    b1_list.extend(b2_list);b1_list.extend(b3_list);b1_list.extend(b4_list)
    b = b1_list
    columns = [1]
    rows = [i for i in range(1,325)]
    b = pd.DataFrame(b,index = rows, columns = columns)
    
    A_list = A.values.tolist()
    b = b.values.tolist()
    FOOTNOTES = list(range(729))
    prob = LpProblem("SudokuSolver", LpMaximize)
    x = LpVariable.dicts("ZeroOneVariables", FOOTNOTES, 0, 1, LpBinary)
    prob += lpSum(x[i] for i in FOOTNOTES)
    for i in range(324):
        prob += lpSum(A_list[i][j] * x[j] for j in FOOTNOTES) == b[i][0]
    prob.solve()
    print(pulp.LpStatus[prob.status])
    if pulp.LpStatus[prob.status] == 'Optimal':
        df_result_list = list()
        for i in range(81):# i is the row of the resulted dataframe
            row_list= list()
            for j in range(9): # j is the column of the resulted dataframe
                row_list.append(x[i * 9 + j].varValue)
            df_result_list.append(row_list)
        len(df_result_list)
        #Now we convert df_result_list into DataFrame
        rows = [i for i in range(1, 82)]
        columns = [i for i in range(1, 10)]
        df_result = pd.DataFrame(df_result_list, index = rows, columns = columns)
        df_final = convert_back(df_result)
        return df_final

In [29]:
import numpy as np
import pandas as pd
columns = [i for i in range(1,10)]
rows = [i for i in range(1,10)]
df = pd.DataFrame([[1,4,0,8,2,0,0,9,5],
              [0,3,0,1,0,0,0,0,0],
              [2,0,0,0,0,9,0,0,7],
              [8,0,2,0,1,4,7,5,3],
              [0,6,0,0,0,5,2,0,0],
              [5,7,0,0,0,0,9,6,1],
              [4,1,9,2,3,8,0,7,6],
              [6,5,3,0,7,0,4,8,2],
              [0,2,8,4,0,0,3,0,9]],index=rows,columns=columns)

In [30]:
final_convert(df)

Optimal


,1,2,3,4,5,6,7,8,9
1,1,4,7,8,2,3,6,9,5
2,9,3,5,1,6,7,8,2,4
3,2,8,6,5,4,9,1,3,7
4,8,9,2,6,1,4,7,5,3
5,3,6,1,7,9,5,2,4,8
6,5,7,4,3,8,2,9,6,1
7,4,1,9,2,3,8,5,7,6
8,6,5,3,9,7,1,4,8,2
9,7,2,8,4,5,6,3,1,9
